In [28]:
%matplotlib inline
import numpy as np
import time 
import matplotlib as mpl
from itertools import product
import matplotlib.pyplot as plt
import pyJHTDB
from pyJHTDB.dbinfo import isotropic1024coarse
from pyJHTDB import libJHTDB

In [6]:
Nx = isotropic1024coarse['nx']; Ny = isotropic1024coarse['ny']; Nz = isotropic1024coarse['nz']
Lx = isotropic1024coarse['lx']; Ly = isotropic1024coarse['ly']; Lz = isotropic1024coarse['lz']
dx = isotropic1024coarse['dx']; dy = isotropic1024coarse['dy']; dz = isotropic1024coarse['dz']

N = Nx

In [26]:
tStep = 0.0
nCubes = 32
m_block = np.array([32,128,512])
displ = np.random.rand(3)

In [27]:
print(m_block*dx)
print(displ)

[ 0.19634954  0.78539816  3.14159265]
[ 0.18108747  0.42969957  0.1540104 ]


In [ ]:
def calcuateVelIncr(u,v,w,dx,f_rStart,f_rEnd,i_rSteps):
    
    long = []
    trans = []
    count = 0
    vel = np.zeros((u.size,3),dtype='float32')
    
    vel[:,:,:,0] = u
    vel[:,:,:,1] = v
    vel[:,:,:,2] = w
    
    incrList = np.logspace(np.log(f_rStart),np.log(f_rEnd),
                           num=i_rSteps,endpoint=True,base=np.e)
    
    for incr in incrList:
        count += 1
        
        for i in range(3):
            
            VV = vel[:,:,:,i]
            velIncr = np.array([])
            
            for j in range(3):
                
                incrSteps = np.floor(incr/dx[j])
                
                for k in range(VV.shape[k]-incrSteps):
                    for j == 1:
                        df = VV[k+incrSteps,:,:]-VV[k,:,:]
                    elif j==2:
                        df = VV[:,k+incrSteps,:]-VV[:,k,:]
                    else
                
    
    
    return long,trans

In [ ]:
begin = time.time()

lJHTDB = libJHTDB()
lJHTDB.initialize()
    
long = []
trans = []

f_rStart = 2*np.pi/N
f_rEnd = 2*np.pi/4
i_rSteps = 40
nCubes = 3
incrList = np.logspace(np.log(f_rStart),np.log(f_rEnd),
                       num=i_rSteps,endpoint=True,base=np.e)

for n in range(nCubes):
    displ = np.random.rand(3)
    block = np.random.permutation(m_block)
    
    indices = np.indices(block)
    
    X = np.zeros((block[0],block[1],block[2],3),dtype='float32')
    Xr = np.zeros((block[0],block[1],block[2],3),dtype='float32')
    X[:,:,:,0] = displ[0]+indices[0]*dx
    X[:,:,:,1] = displ[1]+indices[1]*dy
    X[:,:,:,2] = displ[2]+indices[2]*dz
    
    U = lJHTDB.getData(tStep,X,getFunction = 'getVelocity',
                           data_set = 'isotropic1024coarse')
    
    ############################
    for inc in incrList:
        print("inc = "+str(inc))
        Xr[:,:,:,0] = X[:,:,:,0] + inc
        Xr[:,:,:,1] = X[:,:,:,1]
        Xr[:,:,:,2] = X[:,:,:,2]
        
        Ur = lJHTDB.getData(tStep,Xr,getFunction = 'getVelocity',
                               data_set = 'isotropic1024coarse')
        
        df = Ur - U
        
        lt_df2 = np.average(df[:,:,:,0]**2)
        tv_df2 = np.average(df[:,:,:,1]**2) 
        tv_df2 += np.average(df[:,:,:,2]**2) 
        tv_df2 = tv_df2/2
        long.append(lt_df2)
        trans.append(tv_df2)
    
    ############################
    for inc in incrList:
        print("inc = "+str(inc))
        Xr[:,:,:,0] = X[:,:,:,0]
        Xr[:,:,:,1] = X[:,:,:,1] + inc
        Xr[:,:,:,2] = X[:,:,:,2]
        
        Ur = lJHTDB.getData(tStep,Xr,getFunction = 'getVelocity',
                               data_set = 'isotropic1024coarse')
        
        df = Ur - U
        
        lt_df2 = np.average(df[:,:,:,1]**2)
        tv_df2 = np.average(df[:,:,:,0]**2) 
        tv_df2 += np.average(df[:,:,:,2]**2)
        tv_df2 = tv_df2/2
        long.append(lt_df2)
        trans.append(tv_df2)
    
    ############################
    for inc in incrList:
        print("inc = "+str(inc))
        Xr[:,:,:,0] = X[:,:,:,0]
        Xr[:,:,:,1] = X[:,:,:,1]
        Xr[:,:,:,2] = X[:,:,:,2] + inc
        
        Ur = lJHTDB.getData(tStep,Xr,getFunction = 'getVelocity',
                               data_set = 'isotropic1024coarse')
        
        df = Ur - U
        
        lt_df2 = np.average(df[:,:,:,2]**2)
        tv_df2 = np.average(df[:,:,:,0]**2) 
        tv_df2 += np.average(df[:,:,:,1]**2)
        tv_df2 = tv_df2/2
        long.append(lt_df2)
        trans.append(tv_df2)
    
    ############################
            
    
lJHTDB.finalize()
end = time.time()
print("time = "+str(end-begin))